<font color='green'>***Installation and Libraries Import***</font>
---
--- 

In [1]:
%pip install --upgrade pip
%pip install torch torchvision matplotlib
%pip install async-timeout
%pip uninstall -y flwr ray
%pip install -U "ray>=2.31,<3" "flwr[simulation]>=1.9.0"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Found existing installation: flwr 1.23.0
Uninstalling flwr-1.23.0:
  Successfully uninstalled flwr-1.23.0
Found existing installation: ray 2.31.0
Uninstalling ray-2.31.0:
  Successfully uninstalled ray-2.31.0
Note: you may need to restart the kernel to use updated packages.
  Using cached ray-2.51.1-cp312-cp312-manylinux2014_x86_64.whl.metadata (21 kB)
  Using cached flwr-1.23.0-py3-none-any.whl.metadata (14 kB)
  Using cached ray-2.31.0-cp312-cp312-manylinux2014_x86_64.whl.metadata (13 kB)
Using cached flwr-1.23.0-py3-none-any.whl (731 kB)
Using cached ray-2.31.0-cp312-cp312-manylinux2014_x86_64.whl (66.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [flwr]1/2 [flwr]
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
libraries_to_uninstall = [
    "tb-nightly==2.18.0a20240701",
    "tensorboard==2.16.2",
    "tensorboard-data-server==0.7.2",
    "tensorboard-plugin-wit==1.8.1",
    "tensorflow==2.16.2",
    "tensorflow-io-gcs-filesystem==0.37.0",
    "termcolor==2.4.0",
    "terminado==0.18.1",
    "tf-estimator-nightly==2.8.0.dev2021122109",
    "tf_keras-nightly==2.18.0.dev2024070109",
    "tf-nightly==2.18.0.dev20240626"
]
for library in libraries_to_uninstall:
    os.system(f"pip uninstall -y {library}")

In [3]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, Subset
import torch.optim as optim
from torch.utils.data import Dataset, WeightedRandomSampler
import flwr as fl
from flwr.common import Metrics
# from flwr_datasets import FederatedDataset
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
import random
import math
import pickle
import csv
import copy
print(fl.__version__)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

1.23.0
cuda:0


<font color='Brown'>***FL Constants***</font>
---
--- 

In [4]:
# DEVICE
NUM_CLIENTS = 48
ROUNDS = 40
BATCH_SIZE = 64
LEARNING_RATE = 0.0025
EPOCHS = 3
DATA_GROUPS = 120
BATCH_ROUND = 6
SIZE_ROUND = int(BATCH_ROUND * BATCH_SIZE * NUM_CLIENTS)
PATH = '20241215_3Epochs'

<font color='Light Blue'>***Dataset Preparations***</font>
---
--- 

In [5]:
TrafficData = {}
TrafficData['Dataset']={}
sets_names = ['30','100','70','50','testing']
for  DATA_NUM in sets_names:
    TrafficData['Dataset'][DATA_NUM]=pd.read_csv(f'2_Dataset_5_Attack_{DATA_NUM}_normal.csv', low_memory=False, quoting=csv.QUOTE_NONE, on_bad_lines='skip')
    print(DATA_NUM, TrafficData['Dataset'][DATA_NUM].shape)
for DATA_NUM in TrafficData['Dataset']:
    TrafficData['Dataset'][DATA_NUM]=TrafficData['Dataset'][DATA_NUM].sample(frac=1, random_state=42).reset_index(drop=True)

30 (184320, 99)
100 (184320, 99)
70 (184320, 99)
50 (184320, 99)
testing (120000, 99)


In [6]:
TrafficData['Split'] = {}
sets_training =  ['30','100','70','50']
for DATA_NUM in sets_training:
    TrafficData['Split'][DATA_NUM] = np.array_split(TrafficData['Dataset'][DATA_NUM],DATA_GROUPS)

TrafficData['Combined'] = pd.concat([TrafficData['Split']['30'][0], TrafficData['Split']['100'][0], TrafficData['Split']['70'][0], TrafficData['Split']['50'][0]]).reset_index(drop=True)
for GROUP in range(1, DATA_GROUPS):
    TrafficData['Combined'] = pd.concat([TrafficData['Combined'], TrafficData['Split']['30'][GROUP], TrafficData['Split']['100'][GROUP], TrafficData['Split']['70'][GROUP], TrafficData['Split']['50'][GROUP]]).reset_index(drop=True)
print(TrafficData['Combined'].shape)            

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


(737280, 99)


In [7]:
TrafficData['Train'] = {}
TrafficData['Train']['X'] = TrafficData['Combined'].iloc[:, 0:-1]
TrafficData['Train']['y'] = TrafficData['Combined'].iloc[:, -1]
print(TrafficData['Train']['X'].shape)
print(TrafficData['Train']['y'].shape)

TrafficData['Test'] = {}
TrafficData['Test']['X']=TrafficData['Dataset']['testing'].iloc[:, 0:-1]
TrafficData['Test']['y']=TrafficData['Dataset']['testing'].iloc[:, -1]
print(TrafficData['Test']['X'].shape)
print(TrafficData['Test']['y'].shape)

(737280, 98)
(737280,)
(120000, 98)
(120000,)


In [8]:
scaler = MinMaxScaler()
model = scaler.fit(TrafficData['Train']['X'])
TrafficData['Train']['X'] = model.transform(TrafficData['Train']['X'])
TrafficData['Test']['X'] = model.transform(TrafficData['Test']['X'])

TrafficData['Train']['X'], TrafficData['Train']['y']= np.array(TrafficData['Train']['X']), np.array(TrafficData['Train']['y'])
print(type(TrafficData['Train']['X']),type(TrafficData['Train']['y']))
print(TrafficData['Train']['X'].shape,TrafficData['Train']['y'].shape)
TrafficData['Test']['X'], TrafficData['Test']['y']= np.array(TrafficData['Test']['X']), np.array(TrafficData['Test']['y'])
print(type(TrafficData['Test']['X']),type(TrafficData['Test']['y']))
print(TrafficData['Test']['X'].shape,TrafficData['Test']['y'].shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(737280, 98) (737280,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(120000, 98) (120000,)


In [9]:
TrafficData['ROUNDS']={}
for ROUND in range(1, ROUNDS+1):
    TrafficData['ROUNDS'][ROUND]={}

SIZE_Demo = SIZE_ROUND
for ROUND in range(1,ROUNDS+1):
    if ROUND == 1:
        TrafficData['ROUNDS'][ROUND]['X']= TrafficData['Train']['X'][:SIZE_Demo]
        TrafficData['ROUNDS'][ROUND]['y']= TrafficData['Train']['y'][:SIZE_Demo]
    else:
        print((SIZE_Demo - SIZE_ROUND),SIZE_Demo)
        TrafficData['ROUNDS'][ROUND]['X']= TrafficData['Train']['X'][(SIZE_Demo - SIZE_ROUND):SIZE_Demo]
        TrafficData['ROUNDS'][ROUND]['y']= TrafficData['Train']['y'][(SIZE_Demo - SIZE_ROUND):SIZE_Demo]
    SIZE_Demo = SIZE_Demo + SIZE_ROUND
for ROUND in TrafficData['ROUNDS']:
    print("ROUND: ", ROUND, TrafficData['ROUNDS'][ROUND]['X'].shape, TrafficData['ROUNDS'][ROUND]['y'].shape)
print(SIZE_Demo, SIZE_ROUND)

18432 36864
36864 55296
55296 73728
73728 92160
92160 110592
110592 129024
129024 147456
147456 165888
165888 184320
184320 202752
202752 221184
221184 239616
239616 258048
258048 276480
276480 294912
294912 313344
313344 331776
331776 350208
350208 368640
368640 387072
387072 405504
405504 423936
423936 442368
442368 460800
460800 479232
479232 497664
497664 516096
516096 534528
534528 552960
552960 571392
571392 589824
589824 608256
608256 626688
626688 645120
645120 663552
663552 681984
681984 700416
700416 718848
718848 737280
ROUND:  1 (18432, 98) (18432,)
ROUND:  2 (18432, 98) (18432,)
ROUND:  3 (18432, 98) (18432,)
ROUND:  4 (18432, 98) (18432,)
ROUND:  5 (18432, 98) (18432,)
ROUND:  6 (18432, 98) (18432,)
ROUND:  7 (18432, 98) (18432,)
ROUND:  8 (18432, 98) (18432,)
ROUND:  9 (18432, 98) (18432,)
ROUND:  10 (18432, 98) (18432,)
ROUND:  11 (18432, 98) (18432,)
ROUND:  12 (18432, 98) (18432,)
ROUND:  13 (18432, 98) (18432,)
ROUND:  14 (18432, 98) (18432,)
ROUND:  15 (18432, 98) (

In [10]:
class ClassifierDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
    def __len__ (self):
        return len(self.X_data)

In [11]:
TrafficData['trainsets']={}
for ROUND in range(1, ROUNDS+1):
    TrafficData['trainsets'][ROUND]= ClassifierDataset(torch.from_numpy(TrafficData['ROUNDS'][ROUND]['X']).float(), torch.from_numpy(TrafficData['ROUNDS'][ROUND]['y']).long())
TrafficData['testset'] = ClassifierDataset(torch.from_numpy(TrafficData['Test']['X']).float(), torch.from_numpy(TrafficData['Test']['y']).long())

In [12]:
def load_train(numberofclients, ROUND):    
    portion_size = int(BATCH_ROUND*BATCH_SIZE)
    num_portions = int(NUM_CLIENTS)
    portion_indices = []
    for i in range(num_portions):
        start_idx = i * portion_size
        end_idx = (i + 1) * portion_size
        portion_indices.append(list(range(start_idx, min(end_idx, SIZE_ROUND))))
    portion_datasets = [Subset(TrafficData['trainsets'][ROUND], indices) for indices in portion_indices]
    portion_loaders = [DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False) for dataset in portion_datasets]             
    return portion_loaders
def load_test(numberofclients):    
    testloader = DataLoader(TrafficData['testset'], batch_size=BATCH_SIZE, shuffle=False)
    return testloader

In [13]:
Dataloaders = {}
for ROUND in range(1, ROUNDS+1):
    Dataloaders[ROUND] = load_train(NUM_CLIENTS, ROUND)
Dataloaders['Test'] = load_test(NUM_CLIENTS)

In [14]:
for i, batch in enumerate(Dataloaders['Test']):
    batch_size = len(batch[0])  # Assuming the first element of the batch is the data
    print(f"Batch {i+1} size: {batch_size}")
    if batch_size != 64:
        print(f"Batch {i+1} does not contain 64 records.")
        break

Batch 1 size: 64
Batch 2 size: 64
Batch 3 size: 64
Batch 4 size: 64
Batch 5 size: 64
Batch 6 size: 64
Batch 7 size: 64
Batch 8 size: 64
Batch 9 size: 64
Batch 10 size: 64
Batch 11 size: 64
Batch 12 size: 64
Batch 13 size: 64
Batch 14 size: 64
Batch 15 size: 64
Batch 16 size: 64
Batch 17 size: 64
Batch 18 size: 64
Batch 19 size: 64
Batch 20 size: 64
Batch 21 size: 64
Batch 22 size: 64
Batch 23 size: 64
Batch 24 size: 64
Batch 25 size: 64
Batch 26 size: 64
Batch 27 size: 64
Batch 28 size: 64
Batch 29 size: 64
Batch 30 size: 64
Batch 31 size: 64
Batch 32 size: 64
Batch 33 size: 64
Batch 34 size: 64
Batch 35 size: 64
Batch 36 size: 64
Batch 37 size: 64
Batch 38 size: 64
Batch 39 size: 64
Batch 40 size: 64
Batch 41 size: 64
Batch 42 size: 64
Batch 43 size: 64
Batch 44 size: 64
Batch 45 size: 64
Batch 46 size: 64
Batch 47 size: 64
Batch 48 size: 64
Batch 49 size: 64
Batch 50 size: 64
Batch 51 size: 64
Batch 52 size: 64
Batch 53 size: 64
Batch 54 size: 64
Batch 55 size: 64
Batch 56 size: 64
B

In [15]:
for i, batch in enumerate(Dataloaders[40][0]):
    batch_size = len(batch[0])  # Assuming the first element of the batch is the data
    print(f"Batch {i+1} size: {batch_size}")
    if batch_size != 64:
        print(f"Batch {i+1} does not contain 64 records.")
        break

Batch 1 size: 64
Batch 2 size: 64
Batch 3 size: 64
Batch 4 size: 64
Batch 5 size: 64
Batch 6 size: 64


In [16]:
from collections import Counter
for CLUSTER in range (1, 9):
    DEVICE_PERCENTAGE = []
    for DEVICE__ in range(0, 12):
        for i, batch in enumerate(Dataloaders[CLUSTER][DEVICE__]):
            _, labels = batch
            class_counts = Counter(labels.numpy())
            total_records = sum(class_counts.values())
            class_0_count = class_counts.get(0, 0)
            percentage_class_0 = (class_0_count / total_records) * 100
            DEVICE_PERCENTAGE.append(percentage_class_0)
            # print(f"Batch {i+1}: {dict(class_counts)}")
            # print(f"Percentage of class 0: {percentage_class_0:.2f}%\n")
    # print(DEVICE_PERCENTAGE)        
    chunk_size = 6
    averages = [sum(DEVICE_PERCENTAGE[i:i + chunk_size]) / chunk_size for i in range(0, len(DEVICE_PERCENTAGE), chunk_size)]
    # print("Averages of every device:")
    # print(averages)
    chunk_size_4 = 4
    averages = [sum(averages[i:i + chunk_size_4]) / chunk_size_4 for i in range(0, len(averages), chunk_size_4)]
    # print("Averages of every 4 devices:")
    print(averages)

[29.166666666666668, 100.0, 70.3125]
[31.380208333333336, 100.0, 70.05208333333334]
[29.947916666666668, 100.0, 70.703125]
[29.361979166666668, 100.0, 68.75]
[30.143229166666668, 100.0, 68.88020833333334]
[27.994791666666668, 100.0, 68.88020833333333]
[29.557291666666664, 100.0, 70.703125]
[28.059895833333332, 100.0, 70.05208333333333]


In [17]:
del TrafficData

<font color='Red'>***Neural Network***</font>
---
--- 

In [18]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()        
        self.layer_1 = nn.Linear(98, 64)
        self.layer_2 = nn.Linear(64, 32)
        self.layer_out = nn.Linear(32, 15) 
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.layer_1(x)
        x = self.relu(x)
        x = self.layer_2(x)
        x = self.relu(x)
        x = self.layer_out(x)        
        return x

torch.save(Net().state_dict(), "0_Input_Random_model_Net.pth")

In [19]:
# Random  = Net()
# for param_tensor in Random.state_dict():
#     print(param_tensor, "\t", Random.state_dict()[param_tensor].size())
# torch.save(Random.state_dict(), "0_Input_Random_model_Net.pth")

In [20]:
def train(net, trainloader, epochs: int, verbose=True):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
    net.train()
    prediction_matrix = []
    actual_matrix= []
    acc_matrix = []
    loss_matrix=[]
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
            predictions = torch.max(outputs.data, 1)[1]
            prediction_matrix.append(predictions.tolist())
            actual_matrix.append(labels.tolist())
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        loss_matrix.append(epoch_loss.tolist())
        acc_matrix.append(epoch_acc)
    return prediction_matrix, actual_matrix, acc_matrix, loss_matrix
def test(net, testloader):
    criterion = nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    prediction_matrix = []
    actual_matrix= []
    acc_matrix = []
    loss_matrix=[]
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            prediction_matrix.append(predicted.tolist())
            actual_matrix.append(labels.tolist())
    loss /= len(testloader.dataset)
    accuracy = correct / total
    loss_matrix.append(loss)
    acc_matrix.append(accuracy)    
    print(f"Evaluation: eval loss {loss}, eval accuracy {accuracy}")
    return loss, accuracy, prediction_matrix, actual_matrix, acc_matrix, loss_matrix

In [21]:
prediction_dict= {}
actual_dict= {}
accuracy_dict= {}
loss_dict= {}

<font color='Brown'>***Federated Learning Classes***</font>
---
--- 

In [22]:
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [23]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, FL_Update):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.FL_Update = FL_Update
    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)
    def fit(self, parameters, config):
        local_epochs = config["local_epochs"]
        print(f"[Client {self.cid}, round {self.FL_Update}] fit, config: {config}")
        set_parameters(self.net, parameters)
        _1, _2, _3, _4 = train(self.net, self.trainloader, epochs=local_epochs)
        prediction_dict[f'C{self.cid}R{self.FL_Update}'] = _1
        actual_dict[f'C{self.cid}R{self.FL_Update}'] = _2
        accuracy_dict[f'C{self.cid}R{self.FL_Update}'] = _3
        loss_dict[f'C{self.cid}R{self.FL_Update}'] = _4
        # Save model updates (parameters)
        # update_filename = f'EdgeCooperation/Performance/Results/C{self.cid}R{self.FL_Update}_update.pkl'
        # with open(update_filename, 'wb') as update_outfile:
        #     pickle.dump(get_parameters(self.net), update_outfile)        

        return get_parameters(self.net), len(self.trainloader), {}
    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}")
        print(f"[Client {self.cid}] evaluate, config: {config}")
        return None

In [24]:
def retrieve_and_sort_client_updates(global_model, round_number, client_ids):
    client_updates = {}
    for cid in client_ids:
        update_filename = f'EdgeCooperation/Performance/Results/C{cid}R{round_number}_update.pkl'
        with open(update_filename, 'rb') as update_file:
            client_update = pickle.load(update_file)
            client_updates[cid] = client_update
    client_contributions = {cid: calculate_weight_magnitude(global_model, update) for cid, update in client_updates.items()}
    sorted_clients = sorted(client_contributions.items(), key=lambda x: x[1], reverse=True)
    least_contributing_clients = sorted_clients[-3:]
    return sorted_clients, least_contributing_clients

def calculate_weight_magnitude(global_model, client_update):
    """
    Calculate the L2 norm of the weight difference between the global model and client's updated model.
    
    Args:
    global_model (nn.Module): The global model before client update.
    client_update (list): List of numpy arrays representing client's updated model parameters.

    Returns:
    float: The L2 norm of the weight difference.
    """
    weight_diff = 0.0
    global_parameters = [param.detach().cpu().numpy() for param in global_model.parameters()]
    for global_param, client_param in zip(global_parameters, client_update):
        weight_diff += np.linalg.norm(global_param - client_param)
    return weight_diff


<font color='Brown'>***Clients Functions***</font>
---
--- 

In [25]:
def General_Client():
    def client_fn(cid: int, Round: int) -> FlowerClient:
        clients_ids_list = TrainingListPerRound[Round]
        if int(cid) in clients_ids_list:
            net = Net().to(DEVICE)
            trainloader = Dataloaders[Round][int(cid)]
            arg_ = Round
            return FlowerClient(cid, net, trainloader, arg_)
        else:
            raise ValueError(f"Client ID {cid} not found in the list for round {Round}")
    return client_fn

<font color='Brown'>***FL Strategy***</font>
---
--- 

In [26]:
Global_Models = {}
class SaveModelStrategy(fl.server.strategy.FedAvg):
    def __init__(self, additional_argument, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.additional_argument = additional_argument
    def aggregate_fit(self, rnd: int, results: List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.FitRes]], failures: List[BaseException]) -> Optional[fl.common.NDArrays]:
        aggregated_parameters_tuple = super().aggregate_fit(rnd, results, failures)
        aggregated_parameters, _ = aggregated_parameters_tuple
        if aggregated_parameters is not None:
            # print(f"Saving round {rnd} aggregated_parameters...")
            # Convert `Parameters` to `List[np.ndarray]`
            aggregated_weights: List[np.ndarray] = fl.common.parameters_to_ndarrays(aggregated_parameters)
            # Convert `List[np.ndarray]` to PyTorch`state_dict`
            Global_Models[self.additional_argument] = Net()
            params_dict = zip(Global_Models[self.additional_argument].state_dict().keys(), aggregated_weights)
            state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
            Global_Models[self.additional_argument].load_state_dict(state_dict, strict=True)
            torch.save(Global_Models[self.additional_argument].state_dict(), f"{PATH}/GlobalModel_{self.additional_argument}.pth")
        return aggregated_parameters_tuple

In [27]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.
    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "current_round": server_round,  # The current round of federated learning
        "local_epochs": EPOCHS #1 if rnd < 2 else 2,  # 
    }
    return config

***Running the Generalized FL Round***
---
---

In [28]:
print("Loading Initial Global Model")
Global_Models[0] = Net()
Global_Models[0].load_state_dict(torch.load("0_Input_Random_model_Net.pth"))
Global_Models[0].train()

TrainingListPerRound = {}
for Round in range(1, ROUNDS+1):
    TrainingListPerRound[Round] = []     
    for CLIENT in range (NUM_CLIENTS):
        TrainingListPerRound[Round].append(int(CLIENT))

for Round in range(1, ROUNDS+1):
    print("Starting FL Round: ", Round)
    strategy = SaveModelStrategy(
            fraction_fit=1.0,  # Sample 100% of available clients for training
            fraction_evaluate=0,  # Sample 50% of available clients for evaluation
            min_fit_clients=2,  # Never sample less than 10 clients for training
            min_evaluate_clients=0,  # Never sample less than 5 clients for evaluation
            min_available_clients=2,  # Wait until all 10 clients are available
            on_fit_config_fn=fit_config,
            initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Global_Models[Round-1])),
            additional_argument = Round
    )
    # print(f'Current training nodes at round {Round}: ', len(TrainingListPerRound[Round]), ' ', TrainingListPerRound[Round])
    client_fn = General_Client()
    fl.simulation.start_simulation(
        client_fn=lambda cid: client_fn(cid, int(Round)),
        num_clients=int(NUM_CLIENTS),
        config=fl.server.ServerConfig(num_rounds=int(1)),
        client_resources={"num_cpus":16, "num_gpus":1}, 
        ray_init_args = {'num_cpus': 16, 'num_gpus': 1},
        strategy=strategy
    )
    print("End of FL Round: ", Round)
    print("Loading Global Model: ", Round)
    Global_Models[Round] = Net()
    Global_Models[Round].load_state_dict(torch.load(f"{PATH}/GlobalModel_{Round}.pth"))
    Global_Models[Round].train()

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=1, no round_timeout


Loading Initial Global Model
Starting FL Round:  1


2025-11-03 19:56:01,573	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'CPU': 16.0, 'object_store_memory': 3833863372.0, 'node:10.192.10.115': 1.0, 'memory': 7667726747.0, 'GPU': 1.0, 'accelerator_type:T4': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 16, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 48 clients (out of 48)
(ClientAppActor pid=144602) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(co

(ClientAppActor pid=144602) [Client 4, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}


(ClientAppActor pid=144602) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


...

(ClientAppActor pid=303942) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=303942) 
(ClientAppActor pid=303942)             This is a deprecated feature. It will be removed
(ClientAppActor pid=303942)             entirely in future versions of Flower.
(ClientAppActor pid=303942)         
(ClientAppActor pid=303942) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=303942) [Client 18, round 40] fit, config: {'current_round': 1, 'local_epochs': 3}


INFO :      aggregate_fit: received 48 results and 0 failures
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 1 round(s) in 51.87s
INFO :      


(ClientAppActor pid=303942) [Client 25, round 40] fit, config: {'current_round': 1, 'local_epochs': 3}
End of FL Round:  40
Loading Global Model:  40


(ClientAppActor pid=303942) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=303942) 
(ClientAppActor pid=303942)             This is a deprecated feature. It will be removed
(ClientAppActor pid=303942)             entirely in future versions of Flower.
(ClientAppActor pid=303942)         
(ClientAppActor pid=303942) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


<font color='Grey'>***Performance Testing***</font>
---
--- 

In [29]:
import os
import glob

# Define the directory and file pattern
directory = PATH + '/'
pattern = "GlobalModel_*.pth"

# Find all matching files
files = glob.glob(os.path.join(directory, pattern))

# Extract numbers from file names
numbers = []
for file in files:
    base_name = os.path.basename(file)
    num_str = base_name.replace("GlobalModel_", "").replace(".pth", "")
    try:
        numbers.append(int(num_str))
    except ValueError:
        pass

# Determine the maximum number
max_num = max(numbers) if numbers else 0
print(max_num)

# Use the max_num in a loop
for num in range(1, max_num + 1):
    file_path = f"{PATH}/GlobalModel_{num}.pth"
    if os.path.exists(file_path):
        # Load the file or perform any operation you need
        print(f"Loading {file_path}")
    else:
        print(f"File {file_path} does not exist")

40
Loading 20241215_3Epochs/GlobalModel_1.pth
Loading 20241215_3Epochs/GlobalModel_2.pth
Loading 20241215_3Epochs/GlobalModel_3.pth
Loading 20241215_3Epochs/GlobalModel_4.pth
Loading 20241215_3Epochs/GlobalModel_5.pth
Loading 20241215_3Epochs/GlobalModel_6.pth
Loading 20241215_3Epochs/GlobalModel_7.pth
Loading 20241215_3Epochs/GlobalModel_8.pth
Loading 20241215_3Epochs/GlobalModel_9.pth
Loading 20241215_3Epochs/GlobalModel_10.pth
Loading 20241215_3Epochs/GlobalModel_11.pth
Loading 20241215_3Epochs/GlobalModel_12.pth
Loading 20241215_3Epochs/GlobalModel_13.pth
Loading 20241215_3Epochs/GlobalModel_14.pth
Loading 20241215_3Epochs/GlobalModel_15.pth
Loading 20241215_3Epochs/GlobalModel_16.pth
Loading 20241215_3Epochs/GlobalModel_17.pth
Loading 20241215_3Epochs/GlobalModel_18.pth
Loading 20241215_3Epochs/GlobalModel_19.pth
Loading 20241215_3Epochs/GlobalModel_20.pth
Loading 20241215_3Epochs/GlobalModel_21.pth
Loading 20241215_3Epochs/GlobalModel_22.pth
Loading 20241215_3Epochs/GlobalModel_2

In [30]:
pred_test = {}
actual_test = {}
accuracy_test = {}
loss_test = {}
G = 0

for num in range(1, max_num+1):
    model = Net()
    model.load_state_dict(torch.load(f"{PATH}/GlobalModel_{num}.pth"))
    model.eval()
    
    prediction_matrix = []
    actual_matrix= []
    acc_matrix = []
    loss_matrix=[]
    G = G + 1
    criterion = nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    with torch.no_grad():
        for images, labels in Dataloaders['Test']:
            outputs = model(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            prediction_matrix.append(predicted.tolist())
            actual_matrix.append(labels.tolist())
    loss /= len(Dataloaders['Test'].dataset)
    accuracy = correct / total
    loss_matrix.append(loss)
    acc_matrix.append(accuracy) 

    pred_test[f'Global_{G}'] = prediction_matrix
    actual_test[f'Global_{G}'] = actual_matrix
    accuracy_test[f'Global_{G}'] = acc_matrix
    loss_test[f'Global_{G}'] = loss_matrix 

    filename = f'{PATH}/Global_{G}_pred'
    outfile = open(filename,'wb')
    pickle.dump(pred_test[f'Global_{G}'],outfile)
    outfile.close()

    filename = f'{PATH}/Global_{G}_actual'
    outfile = open(filename,'wb')
    pickle.dump(actual_test[f'Global_{G}'],outfile)
    outfile.close()

    filename = f'{PATH}/Global_{G}_accurracy'
    outfile = open(filename,'wb')
    pickle.dump(accuracy_test[f'Global_{G}'],outfile)
    outfile.close()

    filename = f'{PATH}/Global_{G}_loss'
    outfile = open(filename,'wb')
    pickle.dump(loss_test[f'Global_{G}'],outfile)
    outfile.close()

In [31]:
import pickle
import os

# Directory where your results are stored
path = "20241215_3Epochs"

# Helper function to safely load pickle files
def load_pickle(filepath):
    if os.path.exists(filepath):
        with open(filepath, "rb") as f:
            return pickle.load(f)
    return None

# Loop through all global rounds
for i in range(1, 41):
    print(f"\n========== 🌍 GLOBAL MODEL {i} ==========")
    
    acc_file = os.path.join(path, f"Global_{i}_accurracy")
    loss_file = os.path.join(path, f"Global_{i}_loss")
    pred_file = os.path.join(path, f"Global_{i}_pred")
    actual_file = os.path.join(path, f"Global_{i}_actual")

    accuracy = load_pickle(acc_file)
    loss = load_pickle(loss_file)
    predictions = load_pickle(pred_file)
    actuals = load_pickle(actual_file)

    if accuracy is not None:
        print(f"✅ Accuracy: {accuracy[0]:.4f}" if isinstance(accuracy, list) else f"✅ Accuracy: {accuracy}")
    else:
        print("⚠️ Accuracy file not found")

    if loss is not None:
        print(f"📉 Loss: {loss[0]:.6f}" if isinstance(loss, list) else f"📉 Loss: {loss}")
    else:
        print("⚠️ Loss file not found")

    if predictions is not None and actuals is not None:
        # Display a short sample (not full array)
        print(f"🔢 Predictions sample (first batch): {predictions[0][:10] if len(predictions) > 0 else 'Empty'}")
        print(f"🎯 Actual labels sample (first batch): {actuals[0][:10] if len(actuals) > 0 else 'Empty'}")
        print(f"📦 Total prediction batches: {len(predictions)}")
    else:
        print("⚠️ Predictions or actuals missing")

print("\n✅ All global model files inspected successfully.")



========== 🌍 GLOBAL MODEL 1 ==========
✅ Accuracy: 0.5000
📉 Loss: 0.030527
🔢 Predictions sample (first batch): [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
🎯 Actual labels sample (first batch): [2, 0, 0, 1, 0, 3, 0, 0, 0, 0]
📦 Total prediction batches: 1875

========== 🌍 GLOBAL MODEL 2 ==========
✅ Accuracy: 0.5000
📉 Loss: 0.019134
🔢 Predictions sample (first batch): [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
🎯 Actual labels sample (first batch): [2, 0, 0, 1, 0, 3, 0, 0, 0, 0]
📦 Total prediction batches: 1875

========== 🌍 GLOBAL MODEL 3 ==========
✅ Accuracy: 0.7435
📉 Loss: 0.015174
🔢 Predictions sample (first batch): [2, 0, 0, 0, 0, 0, 0, 0, 0, 0]
🎯 Actual labels sample (first batch): [2, 0, 0, 1, 0, 3, 0, 0, 0, 0]
📦 Total prediction batches: 1875

========== 🌍 GLOBAL MODEL 4 ==========
✅ Accuracy: 0.7261
📉 Loss: 0.013066
🔢 Predictions sample (first batch): [2, 0, 0, 0, 0, 0, 0, 0, 0, 0]
🎯 Actual labels sample (first batch): [2, 0, 0, 1, 0, 3, 0, 0, 0, 0]
📦 Total prediction batches: 1875

========== 🌍 GLOBAL